We take the average across all stations for each of the pollutant and create a mean column to save those average values and then merge all these dataframes to construct a single dataframe with daily(from 2017-2021) pollutant data across all stations and their daily 24h-avg values(from all stations)

In [1]:
import pandas as pd
import numpy as np
import os, glob
from tqdm import tqdm

In [3]:
input_folder = 'merged_air_quality_data_CSV'
output_folder = 'mean_merged_air_quality_CSV'
files = glob.glob(os.path.join(input_folder, '*.csv'))

directory_dict = {'NO2_24h_avg_polution_2017-2021': 'NO2','O3_24h_avg_polution_2017-2021': 'O3','PM10_24h_avg_polution_2017-2021': 'PM10',
            'PM25_24h_avg_polution_2017-2021': 'PM25'}

In [4]:
df_list = []
for f in files:
    #get the file name only: we first split on '/' to get Combined_SD_ECA_snow depth.csv
    #and then leave out the last 4 chars (.csv)
    fname = f.split('/')[1][:-4] 
    print(fname)
    pollutant = directory_dict[fname]
    df = pd.read_csv(f)
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df["date"].dt.strftime("%y-%m-%d") #changing date format to correspond with weather data's DATE column's date format
    df.rename(columns={'date': 'DATE',},inplace=True, errors='raise') #changing date column name to DATE to correspond with weather data's DATE column
    df.set_index('DATE', inplace=True)

    #change the column names to add the pollutant to the source name columns
    df.rename(columns = lambda x: x+'_'+pollutant, inplace = True)

    #create a mean valued col for each row
    mean_df = df.mean(axis=1,skipna=True)
    mean_df = pd.DataFrame(mean_df, columns = ['mean_'+pollutant]) 

    #append the mean column to the pivot table
    mean_added_df = pd.concat([df, mean_df], axis=1)
    #print(mean_added_df.columns)
    df_list.append(mean_added_df)
    print(mean_added_df.shape)
    #break

PM10_24h_avg_polution_2017-2021
(1826, 238)
NO2_24h_avg_polution_2017-2021
(1826, 4)
O3_24h_avg_polution_2017-2021
(1827, 128)
PM25_24h_avg_polution_2017-2021
(1826, 83)


In [5]:
all_concat_df = pd.concat(df_list, axis=1) #concatenate all the weather component dataframes into a single one
all_concat_df.to_csv('./mean_merged_air_quality_CSV/all_mean_air_quality_data_combined.csv')
all_concat_df.shape

(1827, 453)